In [1]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn
from syft import optim

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

In [3]:
import syft

In [6]:
# create remote object with children
x = torch.Tensor([1, 2, 3])
x = syft.LoggingTensor().on(x).send(bob)

# call method on data tensor directly
x.child.point_to_attr = "child.child"
y = x.add(x)
assert isinstance(y.get(), torch.Tensor)

# call method on loggingtensor directly
x.child.point_to_attr = "child"
y = x.add(x)
y = y.get()
assert isinstance(y.child, syft.LoggingTensor)

Log add


In [1]:


# A Toy Dataset
data = torch.tensor([[0,0.],[0,1],[1,0],[1,1]])
target = torch.tensor([[0.],[0],[1],[1]])

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

import torch
from torch.nn.parameter import Parameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules.module import Module
import math

class Linear(Module):
    r"""Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to False, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(N, *, \text{in\_features})` where :math:`*` means any number of
          additional dimensions
        - Output: :math:`(N, *, \text{out\_features})` where all but the last dimension
          are the same shape as the input.

    Attributes:
        weight: the learnable weights of the module of shape
            :math:`(\text{out\_features}, \text{in\_features})`. The values are
            initialized from :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})`, where
            :math:`k = \frac{1}{\text{in\_features}}`
        bias:   the learnable bias of the module of shape :math:`(\text{out\_features})`.
                If :attr:`bias` is ``True``, the values are initialized from
                :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})` where
                :math:`k = \frac{1}{\text{in\_features}}`

    Examples::

        >>> m = nn.Linear(20, 30)
        >>> input = torch.randn(128, 20)
        >>> output = m(input)
        >>> print(output.size())
        torch.Size([128, 30])
    """
    __constants__ = ['bias']

    def __init__(self, in_features, out_features, bias=True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        t = self.weight.t()
        return input.mm(t)
#         return F.linear(input, self.weight, self.bias)

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )

class SGD:
    def __init__(self, params, lr):

        # doing this as a list isn't memory efficient
        self.params = list(params)
        self.lr = lr

    def step(self):

        # TODO: all all the SGD features from PyTorch's SGD
        for p in self.params:
            print(p.location[p.id_at_location])
            p.data -= p.grad# * self.lr
            print(p.location[p.id_at_location])

    def zero_grad(self):
        """We need to use a try/catch because our pointers
        don't always know whether they point to an object
        that exists or not. At the moment, finding out requires
        calling .get() which compromises privacy. There may be
        a more nuanced way to do this in the future."""

        for p in self.params:
            try:
                p.grad -= p.grad
            except:
                ""


# Iniitalize A Toy Model
model = Linear(2,1, bias=False)

bobs_model = model.copy().send(bob)

bobs_opt = SGD(params=bobs_model.parameters(),lr=0.1)

bobs_opt.zero_grad()
bobs_pred = bobs_model(bobs_data)
bobs_loss = ((bobs_pred - bobs_target)**2).sum()
bobs_loss.backward()

p = bobs_opt.params[0]
# p.data -= 2

routing:('OBJ', tensor([[0., 0.],
        [0., 1.]]))
routing:('OBJ', tensor([[0.],
        [0.]]))
routing:('OBJ', tensor([[1., 0.],
        [1., 1.]]))
routing:('OBJ', tensor([[1.],
        [1.]]))
routing:('OBJ', Parameter containing:
tensor([[-0.0775,  0.4842]], requires_grad=True))
disabling gc
18266990651
18266990651
((Wrapper)>[PointerTensor - 68765694695@bob]::grad,)
sending command:('__isub__', [PointerTensor - 68765694695@bob]::grad, ([PointerTensor - 68765694695@bob]::grad,))
id at locatin:68765694695
sending command:('t', [PointerTensor - 68765694695@bob], ())
id at locatin:68765694695
routing:('CMD', (b't', Parameter containing:
tensor([[-0.0775,  0.4842]], requires_grad=True), ()))
execute_command:(b't', Parameter containing:
tensor([[-0.0775,  0.4842]], requires_grad=True), ())
registered response
response from execute_command:[PointerTensor - 27163571729@bob]
before serialization
[PointerTensor - 27163571729@bob]
receiving response:[PointerTensor - 27163571729@bob]
we g

In [ ]:
p.location[p.id_at_location]

In [2]:
p.id_at_location

68765694695

KeyError: 68765694695

In [1]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn
from syft import optim

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
''
x = nn.Parameter(torch.tensor([1.,2,3,4])).send(bob)


routing:('OBJ', Parameter containing:
tensor([1., 2., 3., 4.], requires_grad=True))


In [2]:
# p = x.location[x.id_at_location].data
# message = (b'__iadd__', p, [p])
# bob.execute_command(message)
# x.location[x.id_at_location]

In [3]:
x.data.id_at_location

returning child


82525288441

In [4]:
type(x.location[x.id_at_location])

torch.nn.parameter.Parameter

In [5]:
x.location[x.id_at_location].data

tensor([1., 2., 3., 4.])

In [6]:
x.child.garbage_collect_data = False

In [7]:
x.id_at_location

82525288441

In [8]:
x.data -= 1

returning child
disabling gc
83882526630
32458995603
(1,)
sending command:('__isub__', [PointerTensor - 82525288441@bob]::data, (1,))
id at locatin:82525288441
tensor attr:data id with attr:58917646075
routing:('CMD', (b'__isub__', tensor([1., 2., 3., 4.]), (1,)))
execute_command:(b'__isub__', tensor([1., 2., 3., 4.]), (1,))
skip registration
response from execute_command:[PointerTensor - 58917646075@bob]
before serialization
[PointerTensor - 58917646075@bob]
receiving response:[PointerTensor - 58917646075@bob]
we got a response!:[PointerTensor - 58917646075@bob]
__isub__
about to hook response
FOUND INLINE: returning new self
routing:('OBJ_DEL', 58917646075)


In [9]:
x.id_at_location

82525288441

In [10]:
x.location[x.id_at_location]

Parameter containing:
tensor([0., 1., 2., 3.], requires_grad=True)

In [5]:
x

Parameter containing:
(Wrapper)>[PointerTensor - 87076410322@bob]